In [ ]:
# Import libraries
from datetime import datetime
import glob
import numpy as np
import matplotlib.pyplot as plt
import cmocean
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr
import xskillscore as xs
import hvplot.xarray
import metpy.calc as mpcalc
import metpy.plots as mpplots
from matplotlib.patheffects import withStroke
from metpy.io import parse_metar_file
from metpy.units import pandas_dataframe_to_unit_arrays
from siphon.catalog import TDSCatalog
import re
import dask
import gc
from dask.distributed import Client

In [14]:
# Axis extents for each region in [W, E, S, N] format
# Timezones in hours +- GMT
settings = {
    # Central America (mostly Honduras-Nicaragua-Costa Rica)
    "ca": {"extent": [-91, -81, 7, 17], "tz": -6},
    # South America (mostly eastern Brazil)
    "sa": {"extent": [-65, -30, -15, 0], "tz": -3},
    # Western Australia (mostly near the west coast)
    "wa": {"extent": [113, 123, -35, -30], "tz": +8}
}

In [17]:
def calc_mlai_climatology(region, periodstart, periodend, subset, outputname=None):
    assert(region in settings.keys()), f"region not one of: {*[*settings],}"
    # avhrr vs modis?
    # assert periodstart and periodend structure is mmm-yyyy
    # asset periodstart is after jan-1981 and periodend is before dec-2021
    
    
    # assert periodstart and periodend structure is yyyy-ddd
    # assert periodstart is after 1981-001 and periodend is before 2021-365
    print(region)

In [19]:
calc_mlai_climatology("ca", 1980, 2000, "jja")

ca


In [10]:
areas["wa"]["extent"]

[113, 123, -35, -30]

In [ ]:
# Axis extents for each region in [W, E, S, N] format
extent = {
    # Central America (mostly Honduras-Nicaragua-Costa Rica)
    "ca": [-91, -81, 7, 17],
    # South America (mostly eastern Brazil)
    "sa": [-65, -30, -15, 0],
    # Western Australia (mostly near the west coast)
    "wa": [113, 123, -35, -30]
}

In [ ]:
tz = {"ca": -6, # +- 0
      "sa": -3, # +- 1.5
      "wa": +8} # +- 0